In [58]:
import pandas as pd

file_path = 'data/hcm_weather.csv'
df = pd.read_csv(file_path)

In [49]:
# Số dòng/cột
print("Số dòng:", df.shape[0])
print("Số dòng:", df.shape[1])

Số dòng: 128448
Số dòng: 11


In [50]:
# xem qua dữ liệu
df.head(10)

,latitude,longitude,datetime,temperature (C degree),humidity (%),precipitation (mm),windspeed (km/h),winddirection (degrees),cloudcover (%),pressure (hPa),dewpoint (C degree)
0,10.775388,106.702825,2024-01-01T00:00,25.6,88,0.0,5.7,145,99,1011.5,23.4
1,10.775388,106.702825,2024-01-01T01:00,25.3,90,0.0,3.6,143,96,1011.2,23.5
2,10.775388,106.702825,2024-01-01T02:00,25.3,88,0.0,0.5,135,96,1010.7,23.2
3,10.775388,106.702825,2024-01-01T03:00,24.9,91,0.0,4.7,9,98,1010.5,23.3
4,10.775388,106.702825,2024-01-01T04:00,24.6,92,0.0,8.0,352,100,1010.4,23.2
5,10.775388,106.702825,2024-01-01T05:00,24.4,92,0.0,7.1,345,99,1010.9,23.0
6,10.775388,106.702825,2024-01-01T06:00,24.2,92,0.0,6.3,347,98,1011.2,22.8
7,10.775388,106.702825,2024-01-01T07:00,24.5,90,0.0,5.2,348,98,1011.8,22.7
8,10.775388,106.702825,2024-01-01T08:00,26.2,78,0.0,7.1,345,99,1013.0,22.1
9,10.775388,106.702825,2024-01-01T09:00,28.1,68,0.0,6.3,336,100,1013.6,21.6


In [51]:
# Các cột và định dạng từng cột:
cols = df.columns
print(df.dtypes)

latitude                   float64
longitude                  float64
datetime                    object
temperature (C degree)     float64
humidity (%)                 int64
precipitation (mm)         float64
windspeed (km/h)           float64
winddirection (degrees)      int64
cloudcover (%)               int64
pressure (hPa)             float64
dewpoint (C degree)        float64
dtype: object


Các cột có kiểu dữ liệu phù hợp, chỉ có cột 'datetime' đang là kiểu object thay vì kiểu datetime. Ngoài ra, nhiều dòng có cùng date, khác time cho nên ta tách date time thành 2 cột thuộc tính để dễ sử dụng khi phân tích

In [59]:
# Định dạng datetime từ kiểu object thành kiểu datetime
df['datetime'] = pd.to_datetime(df['datetime'])

# Tách thêm 2 cột date và time
df['date'] = pd.to_datetime(df['datetime'].dt.date, format='%d/%m/%Y')
df['time'] = df['datetime'].dt.strftime('%H:%M:%S')
df = df.drop(columns=['datetime']) # drop cột 'datetime' sau khi tách

# Xem các cột sau khi định dạng và tách
print(df['date'].head())
print(df['time'].head())

0   2024-01-01
1   2024-01-01
2   2024-01-01
3   2024-01-01
4   2024-01-01
Name: date, dtype: datetime64[ns]
0    00:00:00
1    01:00:00
2    02:00:00
3    03:00:00
4    04:00:00
Name: time, dtype: object


In [60]:
# Số dòng thiếu ở mỗi cột:
print(df.isnull().sum())

latitude                   0
longitude                  0
temperature (C degree)     0
humidity (%)               0
precipitation (mm)         0
windspeed (km/h)           0
winddirection (degrees)    0
cloudcover (%)             0
pressure (hPa)             0
dewpoint (C degree)        0
date                       0
time                       0
dtype: int64


In [61]:
# Số dòng trùng lặp
duplicate_count = df.duplicated().sum()
print(f"Số dòng trùng lặp: {duplicate_count}")

Số dòng trùng lặp: 0


In [62]:
# Khoảng giá trị của các cột
numeric_cols = df.select_dtypes(include=['number']).columns # chọn các cột dữ liệu dạng số

for col in numeric_cols:
    if df[col].isna().all(): # cột rỗng --> skip
        continue

    min_val = df[col].min()
    max_val = df[col].max()
    print(f"{col:<25} {min_val} - {max_val}")

latitude                  10.41138 - 10.995955
longitude                 106.511271 - 107.136224
temperature (C degree)    18.5 - 38.9
humidity (%)              23 - 100
precipitation (mm)        0.0 - 24.7
windspeed (km/h)          0.0 - 37.6
winddirection (degrees)   1 - 360
cloudcover (%)            0 - 100
pressure (hPa)            999.9 - 1019.2
dewpoint (C degree)       9.4 - 28.0


--- KHOẢNG GIÁ TRỊ THAM CHIẾU ---
- latitude: -90 đến 90
- longitude: -180 đến 180
- temperature: Mức thấp kỷ lục (-89.2°C) đến Mức cao kỷ lục (56.7°C)
- humidity: 0% đến 100%
- precipitation: >= 0 mm
- windspeed: >= 0 km/h
- winddirection: 0° đến 360°
- cloudcover: 0% đến 100%
- pressure: 870 hPa đến 1085 hPa

So sánh với khoảng giá trị tham chiếu, các thuộc tính đều hợp lý, không có điểm bất thường

In [63]:
# xem xét các giá trị lat/long
locations = df[['latitude', 'longitude']].drop_duplicates()

print(f"Số lượng cặp (lat, long): {len(locations)}")
print("\nChi tiết các tọa độ:")
print(locations)

Số lượng cặp (lat, long): 8

Chi tiết các tọa độ:
         latitude   longitude
0       10.775388  106.702825
16056   10.839823  106.670396
32112   10.995955  106.511271
48168   10.721314  106.546622
64224   10.475805  106.865040
80280   10.411380  107.136224
96336   10.904439  106.782443
112392  10.738544  106.729361


In [ ]:
# tìm những điểm dữ liệu trùng lặp về các điều kiện thời tiết nhưng có giá trị precipitation và toạ độ khác nhau
print("\nCác điểm dữ liệu trùng lặp về các điều kiện thời tiết nhưng có giá trị precipitation và toạ độ khác nhau")
print("(latitude, longitude) : precipitation\n")

check_cols = []
for name in df.columns:
    if name not in ['latitude', 'longitude', 'precipitation (mm)']:
        check_cols.append(name)
        
check_precipitation_df = df.groupby(check_cols)
count = 0 
for name, group in check_precipitation_df:
    if group["precipitation (mm)"].nunique() > 1:
        count += 1

        long = group["longitude"].values
        lat = group["latitude"].values
        prec = group["precipitation (mm)"].values
        for i in range(len(long)):
            print(f"({lat[i]}, {long[i]}): {prec[i]}")
        print()

print(f"Tỉ lệ trùng lặp các điều kiện thời tiết nhưng khác toạ độ và lượng mưa: {count*100/len(check_precipitation_df)}%")


Các điểm dữ liệu trùng lặp về các điều kiện thời tiết nhưng có giá trị precipitation và toạ độ khác nhau
(latitude, longitude) : precipitation

(10.775388, 106.702825): 2.6
(10.738544, 106.729361): 3.6

(10.775388, 106.702825): 3.2
(10.738544, 106.729361): 3.5

(10.839823, 106.670396): 0.6
(10.995955, 106.511271): 0.9

(10.775388, 106.702825): 0.2
(10.738544, 106.729361): 0.1

Tỉ lệ trùng lặp các điều kiện thời tiết nhưng khác toạ độ và lượng mưa: 0.003114343107180118%


Xem xét 2 cột 'latitude' và 'longtitude', nhóm có những cân nhắc sau:

**Vấn đề**

Cần xác định vai trò của latitude và longitude trong việc dự báo mưa (nên giữ hay bỏ 2 cột này?). Liệu sự khác biệt địa lý có ảnh hưởng đến mưa/lượng mưa?

**Quyết định**

- Khi phân tích dữ liệu: không xét latitude và longitude vì chênh lệch lượng mưa giữa các vị trí là không đáng kể

- Khi huấn luyện mô hình: dùng latitude và longitude vì có sự chênh lệch lượng mưa giữa các khu vực, cần thiết để model hoạt động chính xác

In [64]:
df.to_csv('data/hcm_weather_processed.csv', index=False)